# aP Validation Against Log File Example

All communication between PLR and the outside world (the hardware) happens through the io layer. This is a layer below backends, and is responsible for sending and receiving messages to and from the hardware. Schematically,

```
Frontends <-> backends <-> io <-> hardware
```

PLR supports capturing all communication in the io layer, both write and read commands. This can later be played back to validate that a protocol has not changed. The key here is that if we send the same commands to the hardware, the hardware will do the same thing. "Reading" data (from the capture file) is useful because some protocols are dynamic and depend on responses from the hardware.

In this notebook, we will run a simple protocol on a robot while capturing all data passing through the io layer. We will then replay the capture file while executing the protocol again to demonstrate how validation works. Finally, we slightly modify the protocol and show that the validation fails.

## Defining a simple protocol

In [1]:
import pylabrobot
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STAR
from pylabrobot.resources.hamilton import STARDeck

backend = STAR()
lh = LiquidHandler(backend=backend, deck=STARDeck())

In [2]:
from pylabrobot.resources import TIP_CAR_480_A00, HT
tip_car = TIP_CAR_480_A00(name="tip_car")
tip_car[0] = tr = HT(name="ht")
lh.deck.assign_child_resource(tip_car, rails=1)

In [3]:
async def simple_protocol(tips):
  await lh.setup()
  await lh.pick_up_tips(tips)
  await lh.return_tips()
  await lh.stop()

## Capturing data during protocol execution

Do a real run first, without validating against a capture file. This will generate the capture file you can later compare against.

While it might seem cumbersome, this actually ensures you have a real working protocol before doing validation. The benefit of using capture files is whenever you change your protocol and have seen it run, you can just grab the capture file and use it for validation. No need to manually write tests.

In [4]:
validation_file = "./validation.json"
pylabrobot.start_capture(validation_file)
await simple_protocol(tr["A1:H1"])
pylabrobot.stop_capture()

Validation file written to validation.json


The validation file is just json:

In [8]:
!head -n15 validation.json

{
  "version": "0.1.6",
  "commands": [
    {
      "module": "usb",
      "device_id": "[0x8af:0x8000][][]",
      "action": "write",
      "data": "C0RTid0001"
    },
    {
      "module": "usb",
      "device_id": "[0x8af:0x8000][][]",
      "action": "read",
      "data": "C0RTid0001er00/00rt0 0 0 0 0 0 0 0"
    },


## Replaying the capture file for validation

On validation, before calling setup, run `pylabrobot.validate` to enable the validation. Pass a capture file that contains the commands we should check against.

Call `pylabrobot.end_validation` at the end to make sure there are no remaining commands in the capture file. This marks the end of the validation.

In [6]:
pylabrobot.validate(validation_file)
await simple_protocol(tr["A1:H1"])
pylabrobot.end_validation()

Validation successful!


## Failing validation

When validation is not successful, we use the Needleman-Wunsch algorithm to find the difference between the expected and the actual output.

In [7]:
pylabrobot.validate(validation_file)
await simple_protocol(tr["A2:H2"])
pylabrobot.end_validation()

expected: C0TPid0009xp01179 01179 01179 01179 01179 01179 01179 01179yp1458 1368 1278 1188 1098 1008 0918 0828tm1 1 1 1 1 1 1 1tt01tp2266tz2166th2450td0
actual:   C0TPid0009xp01269 01269 01269 01269 01269 01269 01269 01269yp1458 1368 1278 1188 1098 1008 0918 0828tm1 1 1 1 1 1 1 1tt01tp2266tz2166th2450td0
                        ^^    ^^    ^^    ^^    ^^    ^^    ^^    ^^                                                                                    


ValidationError: Data mismatch: difference was written to stdout.